In [ ]:
# Date: 13.02.25
# Note: The plan is to create a system users can access when a customer contacts them for a rate negotiation.
# Design notes:
# - Data: credit policies, competitor insights, past rate offers and reactions
# - Build: No CI/CD/CT atm, just file ingestion and set up. Thinking 
# - - Agent enters cust no triggering SQL query to obtain key cust data (loan amt, tenure, recent engagements, etc.)
# - - Cust data is integrated into a system prompt & run against credit policy to assess if any room for discount (if applicable)
# - - (does this add value?) Query run against vector db finding similar cust who were offered & accepted deal (cust ranked by smallest discount)
# - - Scan of any competitor data (to see if any insights that can aid this conversation/rate neg)


# Wait, to start i just want some SQlite db (cust data, product data, trans data). 
# I want to enter a cust number and be able to pull back info from each of those db


# - - Code can combine inputs from those fields, plus add a system prompt.
# - - Credit policies: need to use certain fields from input to get parameters from credit policies

# - - Three seperate vector DB searchs?
# - - Possibly 
# - - Combine results of the above searches and send to llm with system prompt 

In [1]:
print('fart')

fart


In [13]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect("customer_tbl.db")
cursor = conn.cursor()

# Drop table if it exists (to avoid duplicate data on re-run)
cursor.execute("DROP TABLE IF EXISTS users")

# Create the users table with a manually assigned unique customer_id
cursor.execute("""
CREATE TABLE users (
    customer_id INTEGER PRIMARY KEY,  -- No AUTOINCREMENT, as we assign IDs manually
    name TEXT NOT NULL,
    age INTEGER NOT NULL,
    sex TEXT CHECK(sex IN ('Male', 'Female', 'Other')) NOT NULL,
    tenure INTEGER NOT NULL
)
""")

# Manually assigned customer IDs
users_data = [
    (1001, "Alice Johnson", 30, "Female", 5),
    (1002, "Bob Smith", 45, "Male", 10),
    (1003, "Charlie Brown", 25, "Male", 2),
    (1004, "Diana Ross", 40, "Female", 8),
    (1005, "Ethan Hunt", 35, "Male", 6),
    (1006, "Fiona Davis", 50, "Female", 12),
    (1007, "George Miller", 29, "Male", 3),
    (1008, "Hannah Lee", 33, "Female", 7),
    (1009, "Ian Clark", 28, "Male", 4),
    (1010, "Jessica White", 38, "Female", 9)
]

# Insert data with assigned customer_id
cursor.executemany("INSERT INTO users (customer_id, name, age, sex, tenure) VALUES (?, ?, ?, ?, ?)", users_data)

# Fetch and print the inserted data with customer_id
cursor.execute("SELECT * FROM users")
rows = cursor.fetchall()

# Commit changes and close connection
conn.commit()
conn.close()

print("Database table created and populated successfully!")


Database table created and populated successfully!


In [14]:
# query db
import sqlite3
# Connect to the database
conn = sqlite3.connect("customer_tbl.db")
cursor = conn.cursor()
# Write query in a more MS SQL Server-like style
query = """
SELECT 
    customer_id AS ID, 
    name AS CustomerName, 
    age AS Age, 
    sex AS Gender, 
    tenure AS TenureYears
FROM users
ORDER BY customer_id ASC;
"""

# Execute the query
cursor.execute(query)
rows = cursor.fetchall()
# Print results
for row in rows:
    print(row)
# Close the connection
conn.close()


(1001, 'Alice Johnson', 30, 'Female', 5)
(1002, 'Bob Smith', 45, 'Male', 10)
(1003, 'Charlie Brown', 25, 'Male', 2)
(1004, 'Diana Ross', 40, 'Female', 8)
(1005, 'Ethan Hunt', 35, 'Male', 6)
(1006, 'Fiona Davis', 50, 'Female', 12)
(1007, 'George Miller', 29, 'Male', 3)
(1008, 'Hannah Lee', 33, 'Female', 7)
(1009, 'Ian Clark', 28, 'Male', 4)
(1010, 'Jessica White', 38, 'Female', 9)


In [2]:
import sqlite3

# Connect to SQLite (or create the database file)
conn = sqlite3.connect("bank_data.db")
cursor = conn.cursor()

# Create tables
cursor.execute('''
CREATE TABLE IF NOT EXISTS customers (
    customer_id INTEGER PRIMARY KEY,
    name TEXT,
    age INTEGER,
    location TEXT,
    tenure INTEGER
);
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS products (
    customer_id INTEGER,
    product TEXT,
    description TEXT,
    rate REAL,
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
);
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS transactions (
    customer_id INTEGER,
    date TEXT,
    trans TEXT,
    amount REAL,
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
);
''')

# Insert data into customers
data_customers = [
    (1, "Alice Johnson", 34, "Sydney", 10),
    (2, "Bob Smith", 45, "Melbourne", 15),
    (3, "Charlie Brown", 29, "Brisbane", 5)
]
cursor.executemany("INSERT INTO customers VALUES (?, ?, ?, ?, ?)", data_customers)

# Insert data into products
data_products = [
    (1, "Home Loan", "30-year fixed mortgage", 3.5),
    (2, "Credit Card", "Platinum rewards card", 18.0),
    (3, "Savings Account", "High-interest savings", 2.0)
]
cursor.executemany("INSERT INTO products VALUES (?, ?, ?, ?)", data_products)

# Insert data into transactions
data_transactions = [
    (1, "2025-01-15", "Loan Payment", -2000.00),
    (2, "2025-01-20", "Credit Card Purchase", -150.75),
    (3, "2025-01-22", "Deposit", 500.00)
]
cursor.executemany("INSERT INTO transactions VALUES (?, ?, ?, ?)", data_transactions)

# Commit changes and close connection
conn.commit()
conn.close()

print("Database created and populated successfully.")


Database created and populated successfully.


In [18]:
#pip install tabulate
from tabulate import tabulate

In [19]:
# -- Query db --
import sqlite3
# Connect to the database
conn = sqlite3.connect("bank_data.db")
cursor = conn.cursor()

# 1.0 Customer query
query = """
SELECT 
    customer_id AS ID, 
    name AS CustomerName, 
    age AS Age, 
    tenure AS TenureYears
FROM customers
ORDER BY customer_id ASC;
"""
# Execute the query
cursor.execute(query)
rows = cursor.fetchall()
# Print results
for row in rows:
    print(row)
# Close the connection
#conn.close()


print('\n','-'*5, 'new', '-'*5)
# 2.0 Product query
query2 = """
SELECT 
    customer_id AS ID, 
    product,
    description,
    rate
FROM products
ORDER BY customer_id ASC;
"""
# Execute the query
cursor.execute(query2)
rows = cursor.fetchall()
# Print results
for row in rows:
    print(row)
# Close the connection
#conn.close()


print('\n','-'*5, 'new', '-'*5)
# 3.0 Transaction query
query3 = """
SELECT 
    customer_id AS ID, 
    date,
    trans,
    amount
FROM transactions
ORDER BY customer_id ASC;
"""
# Execute the query
cursor.execute(query3)
rows = cursor.fetchall()
# Print results
for row in rows:
    print(row)
# Close the connection
#conn.close()


print('\n','-'*5, 'new', '-'*5)
# 4.0 Join query
query4 = """
SELECT 
    a.customer_id AS ID, 
    a.name AS CustomerName, 
    a.age AS Age, 
    a.tenure AS TenureYears,
    b.product,
    b.description
FROM customers a
LEFT JOIN products b
  ON a.customer_id = b.customer_id
ORDER BY a.customer_id ASC;
"""
# Execute the query
# Execute the query
cursor.execute(query4)
rows = cursor.fetchall()
# Get column names
headers = [desc[0] for desc in cursor.description]
# Print formatted table
print(tabulate(rows, headers=headers, tablefmt="grid"))

conn.close()




(1, 'Alice Johnson', 34, 10)
(2, 'Bob Smith', 45, 15)
(3, 'Charlie Brown', 29, 5)

 ----- new -----
(1, 'Home Loan', '30-year fixed mortgage', 3.5)
(2, 'Credit Card', 'Platinum rewards card', 18.0)
(3, 'Savings Account', 'High-interest savings', 2.0)

 ----- new -----
(1, '2025-01-15', 'Loan Payment', -2000.0)
(2, '2025-01-20', 'Credit Card Purchase', -150.75)
(3, '2025-01-22', 'Deposit', 500.0)

 ----- new -----
+------+----------------+-------+---------------+-----------------+------------------------+
|   ID | CustomerName   |   Age |   TenureYears | product         | description            |
+======+================+=======+===============+=================+========================+
|    1 | Alice Johnson  |    34 |            10 | Home Loan       | 30-year fixed mortgage |
+------+----------------+-------+---------------+-----------------+------------------------+
|    2 | Bob Smith      |    45 |            15 | Credit Card     | Platinum rewards card  |
+------+----------------+

In [ ]:
# Checking llm call (not part of code)
import os
from typing import List
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
#from openai import OpenAI

load_dotenv(override=True)
api_key = os.getenv('GOOGLE_API_KEY')
import os
import google.generativeai as genai
genai.configure(api_key= api_key)

# Create the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
  #"response_mime_type": "application/json",
}

model = genai.GenerativeModel(model_name="gemini-1.5-flash",
  generation_config=generation_config,)
chat_session = model.start_chat(
  history=[  ])
message = "this is a test only"
response = chat_session.send_message(message)
print(response.text)